<a href="https://colab.research.google.com/github/sohelshekhatik1998/my-all-work/blob/main/Copy_of_Data_grokr__chess_data_analysis_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##SECTION-1

In [1]:
!apt-get update -y

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [5]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [6]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

^C


In [7]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz


gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now


In [8]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [10]:
!pip install -q findspark
import findspark
findspark.init()

Exception: ignored

In [ ]:
from pyspark.sql import SparkSession
spark = (SparkSession
.builder
.appName("<app_name>")
.getOrCreate())

In [ ]:
!pip install gdown

In [ ]:
import gdown

# a file
url = "https://drive.google.com/drive/folders/1QgWPHV_l25Ui9L7et8mkZohAOG59UTkQ"
output = "fcn8s_from_caffe.npz"
gdown.download(url, output, quiet=False)

In [ ]:
# a folder
url = "https://drive.google.com/drive/folders/1QgWPHV_l25Ui9L7et8mkZohAOG59UTkQ"
gdown.download_folder(url, quiet=True, use_cookies=False)

In [ ]:
id = "15uNXeRBIhVvZJIhL4yTw4IsStMhUaaxl"
gdown.download_folder(id=id, quiet=True, use_cookies=False)

##SECTION-2

In [ ]:
import cv2 as cv
from google.colab.patches import cv2_imshow
from skimage import io
import pandas as pd

In [ ]:
img = io.imread('/content/chess/chess_schema.png')
cv2_imshow(img)

In [ ]:
#groupedDF = df.groupby('result',sort=False)['event',"white_first_name"].count()
#sortedDF=groupedDF.sort_values('black', ascending=False)['event']
#print(groupedDF)
#df2=df.loc[(df.result== '0-1') | (df.result=='1-0')] 
#df2.head()

In [ ]:
game_info = pd.read_csv('/content/chess/chess_wc_history_game_info.csv')

game_info.head()


In [ ]:
eco_df = pd.read_csv("/content/chess/eco_codes.csv")
eco_df.head()

In [ ]:
moves_df = pd.read_csv("/content/chess/chess_wc_history_moves.csv")
moves_df.head()

In [ ]:
game_info.info()

In [ ]:
game_info.tournament_name.nunique()

In [ ]:
# convert to datetime

# df.date_played = pd.to_datetime(df.date_played)
game_info.date_created = pd.to_datetime(game_info.date_created)
# data played has missing values, dropping it is better
game_info = game_info.drop(columns=['date_played'])


In [ ]:
game_info[['white_result', 'black_result']] = game_info['result'].str.split('-', expand=True)
game_info[['white_first_name','w_last_name']] = game_info['white'].str.split(',', expand=True)
game_info[['black_first_name', 'b_last_name']] = game_info['black'].str.split(",", expand=True)
game_info["white"] = game_info.white_first_name + "-" + game_info.white_result
game_info["black"] = game_info.black_first_name + "-" + game_info.black_result
game_info.head()

### Marge dataset

In [ ]:
# merge 1
df = pd.merge(game_info, moves_df, how='left', on='game_id')

# merge2
df = pd.merge(df, eco_df, how='left', on='eco')
df.head()

### Q.1. List of winners of each world champions Trophy

In [ ]:

winners_per_tournament = pd.DataFrame(game_info.groupby(['tournament_name','winner']).winner.count()).rename(columns={'winner':'score'}).reset_index()
winners_per_tournament = winners_per_tournament[winners_per_tournament.winner != 'draw']
winners_per_tournament = winners_per_tournament.loc[winners_per_tournament.groupby(['tournament_name'])['score'].idxmax()]
winners_per_tournament

### Q. List of Players with number of times they have won Tournament in descending order(Max to min).

Result attributes: player_name, number_of_wins

In [ ]:
winners_per_tournament.sort_values('score', ascending=False)[['winner', 'score']]

### Q.3. Most and Least Popular eco move in world championship history.

In [ ]:
eco_moves = pd.DataFrame(game_info.groupby('eco').eco.count()).rename(columns={'eco':'count'}).reset_index()

# most popular 
print("Most popular moves:")
print(eco_moves.sort_values('count', ascending=False).head(1))
print()

# least popular
print("Least popular moves")
print(eco_moves.sort_values('count', ascending=True).head(1))

### Q.4. Find the eco move with most winnings.

In [ ]:
eco_move_win = pd.DataFrame(game_info[game_info.winner != 'draw'].groupby(['eco']).winner.count()).rename(columns={'winner':'wins'})
eco_move_win = pd.merge(eco_move_win, eco_df[['eco','eco_name']], how='left', on='eco')
eco_move_win.max()

### Q.5. Longest and shortest game ever played in a world championship in terms of move.

In [ ]:
game_time = pd.DataFrame(df.groupby(['game_id', 'event', 'tournament_name']).move_no.max()).rename(columns={'move_no':'number_of_moves'}).reset_index()
print("Longest game ever played: ")
game_time.max()

In [ ]:
print("Shortest game ever played: ")
game_time.min()

### Q.6. Shortest and Longest Draw game ever played

In [ ]:
draw_games = df[df.winner=='draw']
draw_games_time = pd.DataFrame(draw_games.groupby(['game_id','event', 'tournament_name']).move_no.count()).rename(columns={'move_no':'number_of_moves'}).reset_index()

print("Longest draw game ever played: ")
print(draw_games_time.max())
print()

print("Shortest draw game ever played: ")
print(draw_games_time.min())


### Q.7. Most and Least rated Player

In [ ]:
# Most rated player:
print("Most rated player: ")
winner_elo = pd.DataFrame(game_info[game_info.winner != 'draw'].groupby(['winner']).winner_elo.sum()).reset_index().rename(columns={'winner':'player_name', 'winner_elo':'elo'})
winner_elo.max()

In [ ]:
# Least rated player
print("Least rated player: ")
least_rated_player = pd.DataFrame(game_info[game_info.winner != 'draw'].groupby(['loser']).loser_elo.sum()).reset_index().rename(columns={'loser':'player_name', 'loser_elo':'elo'})
least_rated_player.min()

### Q.8. 3rd Last Player with most loss

In [ ]:
loss_df = game_info[game_info.winner !='draw'].groupby(['loser']).loser.count().sort_values()
loss_df[2:3]

### Q.9. How many times players with low rating won matches with their total win Count.

In [ ]:
# Number of wins
win_count = pd.DataFrame(game_info[game_info.winner!='draw'].groupby(['winner']).winner.count().sort_values()).rename(columns={'winner':'win_count'}).reset_index().rename(columns={'winner':'player_name'})

# merge win count with win elo
win_elo_df = pd.merge(winner_elo, win_count, how='left', on='player_name')

# win of lowest rating players
win_elo_df.loc[win_elo_df.elo == win_elo_df.elo.min()]

### Q.10. Move sequence for each player in a match

In [ ]:
move_sequence_df = pd.pivot_table(data=moves_df, index=['game_id','player'], columns=['move_sequence'], values=['move_no'], aggfunc='max')
# move_sequence_df.head()

### Q.11. Total number of games where losing player has more captured score than winning player.
Hint: Captured score is cumulative, i.e., for 3rd capture it will have score for 1, 2, and 3rd.
Result attributes: total_number_of_games Final result will have only one row

In [ ]:
game_info.head()

In [ ]:
def result(row):
    if len(row) == 1:
        return int(row)
    else:
        x,y = row.split('/')
        res = int(x)/int(y)
        return res


game_info.black_result = game_info.black_result.apply(result)
game_info.white_result = game_info.white_result.apply(result)

In [ ]:
white_result = game_info.groupby(['game_id','white_first_name']).white_result.sum()
black_result = game_info.groupby(['game_id','black_first_name']).black_result.sum()


### Q.12. List All Perfect Tournament with Winner Name.

Chess Funda: Perfect Tournament means a player has won all the matches excluding draw matches. e.g Player A has won 5 matches out of 7
Matches in tournament where 2 matches are draw and player B has won 0 matches)
Result attributes: winner_name, tournament_name

In [ ]:
game_info['winner_name','tournament_name'].value_count()

### Q.13. Player with highest winning ratio.
Hint: Winning ratio: (Number of rounds won)/(Number of rounds played)
Result attributes: player_name
Final result will have only one row

In [ ]:
game_info["w_full_name"] = game_info.white_first_name + ' ' + game_info.w_last_name
game_info['b_full_name'] = game_info.black_first_name + ' ' + game_info.b_last_name

In [ ]:
number_of_rounds_played_by_white = pd.DataFrame(game_info.groupby('w_full_name').w_full_name.count()).rename(columns={'w_full_name':'count'}).reset_index().rename(columns={'w_full_name':'player_name'})
number_of_rounds_played_by_black = pd.DataFrame(game_info.groupby('b_full_name').b_full_name.count()).rename(columns={'b_full_name':'count'}).reset_index().rename(columns={'b_full_name':'player_name'})
# number_of_players_per_player = number_of_rounds_played_by_white.add(number_of_rounds_played_by_black)
# number_of_players_per_player
s1 = number_of_rounds_played_by_black.set_index('player_name')['count']
s2 = number_of_rounds_played_by_white.set_index('player_name')['count']
number_of_players_per_player = s1.add(s2, fill_value=0).reset_index(name='count')
number_of_players_per_player.head()

**Names need to be cleaned.**

In [ ]:
# number of wins
number_wins = pd.DataFrame(game_info[game_info.winner !='draw'].groupby(['winner']).winner.count()).rename(columns={'winner':'count'}).reset_index().rename(columns={'winner':'player_name'})
# number_wins.player_name = number_wins.player_name.apply(lambda x: x.split(',')[0])
number_wins['first_name'] = number_wins.player_name.apply(lambda x: x.split(',')[0])
# number_wins['last_name'] = number_wins.player_name.apply(lambda x: x.split(',')[1])
number_wins.head()


### Q.14. Player who had given checkmate with Pawn.
Note: Consider all events for this query
Result attributes: player_name
Final result will have only one row

### Q.15. List games where player has won game without queen.
Result attributes: game_id, event, player_name